In [ ]:
### TODOs ###

# TODOs: Mine Args
# TODO: Enhance Stance Module; Determine stance over entire argument. Only implicate stance for Noun
# TODOs: Mine Counters
# TODOs: Add Concepts
# TODOs: Commonsense Query and Concept Expansion: Topics, Concepts, Synonyms
# TODOs: Parallel process
# TODOs: Prior tokenization and sentence segmentation to speed processing
# TODOs: Domain Restrict. Polarising social and political debate (Class labelling) only for higher-quality argument-knowledge set

In [ ]:
# TODOs: Adu, Counter + KP Extraction as 'Argument Mining' preprocessing module
# TODOs: Implement Query Expansion at Query-time
# TODOs: Manage Duplicate Keywords
# DONE: Sentential Ranking
# DONE: Include Topic Label
# DONE: Include Concept Label
# DONE: Add News
# TODOs: Targeted Retreival with Semantic Graphs
# TODOs: Target Argumentative Content Only
# TODOs: Targeted Argument Content: Adus + Extractive Summary
# TODOs: Query Expansion
# TODOs: Multi-Field Search
# TODOs: Additional News and Knowledge Sources

In [1]:
### INIT LOGGING ###
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ARGUMENT-EXTRACTOR")

### NLP FUNCTIONS ###
from src.utils_.utils import tokeniser, sentences_segment

In [2]:
### LOAD DATASETS ###
import json
import random

args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
topics = [json.loads(ln) for ln in open("../data/argument_topic_concept.jsonl")]
concepts = [json.loads(ln) for ln in open("../data/argument_concept.jsonl")]

In [3]:
len(topics), len(concepts), len(args)

(5990, 5990, 10303)

In [4]:
### ASSERT BLANKS ###
args_ = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]
ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]

for j, k in zip(args_, ids):
    if j == "":
        print("blanks", j, k)

blanks  t3_3cm6jy
blanks  t3_1egv4k
blanks  t3_1egv4k
blanks  t3_5wjdve


In [5]:
### INSPECT ARG ###
import random
sample = random.randint(0, 99)

arg = args[sample]["argument"]["argument"]
claim = args[sample]["claim"]

print(sample, "\n")
print(claim, "\n")
print(arg, "\n")

66 

us sport leagues should create minor league affiliates instead of using college as their scouting ground 

what i mean is to make it into the nfl or nba you essentially have to go to college. you play in high school and hope to god a college scout sees you and picks you up. you play in college and hope to god a pro scout sees you and picks you up. theres no other real way to make it into those leagues. what it does is causes a bunch of people with no desire for college to go to college for the wrong reasons. they dont care about furthering their education or anything that theyre just there in hopes a scout sees them. this seems like it would only negatively effect the education of the rest of the students as part of their classmates have no desire to be there for anything related to education.instead these leagues should have a large affiliate network of professional minor leagues they use like how the nhl does it.they have a draft once a year for everyone years old and older. if 

In [6]:
### EXTRACTORS ###
from src.utils_.keyphrase_extraction import yake_extract_keyphrase, summa_extract_keyphrase

test = "Brazil's minimum income has increasingly been accepted."
ev_kp = yake_extract_keyphrase(test)
ev_kp_ = summa_extract_keyphrase(test)

test_2 = " "
ev_kp_2 = yake_extract_keyphrase(test_2)
ev_kp_2_ = summa_extract_keyphrase(test_2)

print(ev_kp)
print(ev_kp_)

# Can Handel Blanks
print(ev_kp_2)
print(ev_kp_2_)

### KEYBERT ###
# from keybert import KeyBERT
#
# kb = KeyBERT()
# def extract_keyphrase(doc, n_gram=3, n_kp=3, use_mmr="False", use_maxsum="False"):
#     kp = kb.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', use_mmr=True, diversity=0.5)
#
#     return [i[0] for i in kp]

INFO:summa.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
INFO:KEYPHRASE_EXTRACTOR:[Test Keyphrase: ] 
 ['heathrow airport', 'environmental impact', 'aviation']


['Brazil minimum income', 'minimum income has increasingly', 'Brazil minimum', 'increasingly been accepted', 'minimum income']
['minimum']
[]
[]


In [7]:
from tqdm.notebook import tqdm
from src.utils_.word_net_expansion import expand_query

import multiprocessing
import json
import time

# Disable Huggingface Logging
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

topic_ids = [json.loads(ln)["id"] for ln in open("../data/argument_topic_concept.jsonl")]
concept_ids = [json.loads(ln)["id"] for ln in open("../data/argument_concept.jsonl")]

path = "/Users/joshua.sheppard/PycharmProjects/countaBot/src/retriever/"
os.chdir(path)
from src.detection.stance_classifier import sentence_stance
from src.detection.stance_classifier import sentence_stance, compare_stance

# Where notion == topic or concept
def get_notion(notions_ids, notions_lst, arg_id, label):
    notion_id = notions_ids.index(arg_id)
    notion = notions_lst[notion_id][label]
    return str(notion) if notion else None

# Extract Argument Discourse as Sentences, Keyphrases, Topics and Concepts
def extract_adus(arg_):

    id_ = arg_["id"]
    arg = arg_["argument"]["argument"]

    topic = get_notion(topic_ids, topics, id_, "topic_label")
    concept = get_notion(concept_ids, concepts, id_, "concept_label")

    adu_sents = sentences_segment(arg)

    extract_adus = []
    for _ in adu_sents:

        if len(tokeniser(_)) <= 5:
            continue

        #kp = extract_keyphrase(_)
        kp = yake_extract_keyphrase(_)

        aspect = " " if kp == [] else kp[0]

        try:
            stance = sentence_stance(_, aspect)
        except:
            stance = " "

        adu = {"sentence": _, "kp": [i.lower() for i in kp], "stance": stance, "aspect": aspect, "topic": topic, "concept": concept}

        extract_adus.append(adu)

    claim = arg_["claim"]
    claim_kp = yake_extract_keyphrase(claim)
    claim_adu = {"sentence": claim, "kp": [i for i in claim_kp]}

    return ({
        "id": id_,
        "claim": claim_adu,
        "argument": [i for i in extract_adus]
    })

#SAMPLE = args[0:1000]

# SAMPLE = args
# STEPS = 10
# STEP = max(int(len(SAMPLE) / STEPS), 1)
# BATCHES = [sample[i:i + STEP] for i in range(0, len(SAMPLE), STEP)]
#
# mined_args = []
# for idx, batch in enumerate(BATCHES):
#     print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(BATCHES)) + '-' * 25)
#
#     with multiprocessing.Pool(8) as pool:
#         with tqdm(total=(len(batch))) as pbar:
#             for arg in batch:
#                 mined_args.append(extract_adus(arg))
#                 pbar.update()

mined_args = []

# sample = args
# with tqdm(total=(len(sample)), leave=True, position=0) as pbar:
#     for arg in sample:
#         mined_args.append(extract_adus(arg))
#         pbar.update()

with multiprocessing.Pool(8) as pool:
    with tqdm(total=(len(args)), position=0, leave=True) as pbar:
        for arg in args:
            mined_args.append(extract_adus(arg))
            pbar.update()

INFO:STANCE_CLASSIFIER:[Initialised ... ]
INFO:STANCE_CLASSIFIER:[Test Stance ... ] 
 The mutual trust and understanding you share with your partner will lead to better sex, but that's not the only reason sex can be better when you're in a relationship., PRO, better sex


  0%|          | 0/10303 [00:00<?, ?it/s]

In [8]:
# mined_args
print(len(mined_args))

import random
#_ = random.randint(0, len(sample))
example = mined_args[1]
example

10303


{'id': 't3_30oi71',
 'claim': {'sentence': 'we should strengthen the traditional safety net rather than replace it with basic income',
  'kp': ['strengthen the traditional safety',
   'traditional safety net',
   'basic income',
   'strengthen the traditional',
   'traditional safety']},
 'argument': [{'sentence': 'section i why is basic income increasingly popular?',
   'kp': ['basic income increasingly popular',
    'income increasingly popular',
    'basic income increasingly',
    'increasingly popular',
    'basic income'],
   'stance': 'PRO',
   'aspect': 'basic income increasingly popular',
   'topic': 'incomes policy',
   'concept': 'increase in accumulated other comprehensive income'},
  {'sentence': 'basic income is a policy that has broad support from both the progressive left and libertarian right.',
   'kp': ['progressive left and libertarian',
    'basic income',
    'income is a policy',
    'policy that has broad',
    'broad support'],
   'stance': 'PRO',
   'aspect': 

In [9]:
# STORE DEEP-COPY
import copy
mined_args_ = copy.deepcopy(mined_args)

In [10]:
len(mined_args_)

10303

In [12]:
### COUNTER-ARGS ###
def extract_counters(arg_):
    id_ = arg_["id"]
    counter = arg_["tgt_counter"]["tgt_counter"]

    counter_sents = sentences_segment(counter)

    extract_counters = []
    for _ in counter_sents:
        if len(tokeniser(_)) <= 5:
            continue

        #kp = extract_keyphrase(_)
        kp = yake_extract_keyphrase(_)

        aspect = " " if kp == [] else kp[0]

        try:
            stance = sentence_stance(_, aspect)
        except:
            stance = " "

        counter_unit = {"sentence": _, "kp": [i for i in kp], "stance": stance, "aspect": aspect}

        extract_counters.append(counter_unit)

    return ({
        "id": id_,
        "tgt_counter": [i for i in extract_counters]
    })

# STEPS = 10
# STEP = max(int(len(SAMPLE) / STEPS), 1)
# BATCHES = [sample[i:i + STEP] for i in range(0, len(SAMPLE), STEP)]
#
# mined_counters = []
# for idx, batch in enumerate(BATCHES):
#     print('-' * 25 + 'Batch %d/%d' % (idx + 1, len(BATCHES)) + '-' * 25)
#
#     with multiprocessing.Pool(8) as pool:
#         with tqdm(total=(len(batch))) as pbar:
#             for counter in batch:
#                 mined_counters.append(extract_counters(counter))
#                 pbar.update()
mined_counters = []

sample = args
with multiprocessing.Pool(8) as pool:
    with tqdm(total=(len(args)), position=0, leave=True) as pbar:
        for arg in args:
            mined_counters.append(extract_counters(arg))
            pbar.update()

  0%|          | 0/10303 [00:00<?, ?it/s]

In [13]:
# mined_args
print(len(mined_counters))

import random
_ = random.randint(0, len(sample))
example = mined_counters[_]
example

10303


{'id': 't3_2htk52',
 'tgt_counter': [{'sentence': 'commercial dry dog food is full of additives preservatives rendered animal products meat not inspected as safe for humans etc.',
   'kp': ['commercial dry dog food',
    'additives preservatives rendered animal',
    'preservatives rendered animal products',
    'rendered animal products meat',
    'commercial dry dog'],
   'stance': 'PRO',
   'aspect': 'commercial dry dog food'},
  {'sentence': 'some people think dog kibble isnt even healthy for the dog some of the chemicals theyre worried about include bht and ethoxyquin.',
   'kp': ['include bht and ethoxyquin',
    'kibble isnt even healthy',
    'chemicals theyre worried',
    'theyre worried about include',
    'worried about include bht'],
   'stance': 'PRO',
   'aspect': 'include bht and ethoxyquin'},
  {'sentence': 'if the child ate one or two pieces once thats not the end of the world.',
   'kp': ['child ate', 'end of the world', 'world', 'child', 'ate'],
   'stance': 'NEUTRA

In [17]:
len(mined_args), len(mined_counters)

(10303, 10303)

In [15]:
import copy
mined_counters_ = copy.deepcopy(mined_counters)

In [16]:
len(mined_args_), len(mined_counters_)

(10303, 10303)

In [19]:
import os
print(os.getcwd())

/Users/joshua.sheppard/PycharmProjects/countaBot


In [20]:
file_name = "cmv_argument_extraction"
fout = open(f"./src/data/{file_name}.jsonl", "w")

# Deep_copies
mined_args_ = copy.deepcopy(mined_args)
mined_counters_ = copy.deepcopy(mined_counters)

with tqdm(total=(len(mined_args_))) as pbar:
    with fout:
        for mined_arg, mined_counter in zip(mined_args_, mined_counters_):
            # Extended pre-formatted mined object
            # mined_arg["original_post"] = original_post["argument"]
            mined_arg["tgt_counter"] = [_ for _ in mined_counter["tgt_counter"]]

            fout.write(json.dumps(mined_arg))

            fout.write("\n")
            pbar.update()

logger.info(f"[{len(mined_args_)} Data Stored as {file_name}.jsonl]")

  0%|          | 0/10303 [00:00<?, ?it/s]

INFO:ARGUMENT-EXTRACTOR:[10303 Data Stored as cmv_argument_extraction.jsonl]


In [21]:
### EVALUATE OUTPUT ###
train = [json.loads(ln) for ln in open(f"../data/{file_name}.jsonl", "r")]

In [22]:
len(train)

10303

In [28]:
_ = random.randint(0, len(train))
print(train[_]["id"], "\n")
print(train[_]["argument"], "\n")
print(train[_]["tgt_counter"], "\n")

t3_2dqwmd 

[{'sentence': 'Let me start off by saying that Im a gay guy.', 'kp': ['gay guy', 'guy', 'start', 'gay'], 'stance': 'NEUTRAL', 'aspect': 'gay guy', 'topic': None, 'concept': 'gay guy'}, {'sentence': 'Im not really viewing this from the outside in though perhaps that perspective is what I need.Im nineteen years old now coming to realization with my sexuality when I had just turned thirteen years old.', 'kp': ['need.Im nineteen years', 'turned thirteen years', 'need.Im nineteen', 'coming to realization', 'turned thirteen'], 'stance': 'NEUTRAL', 'aspect': 'need.Im nineteen years', 'topic': None, 'concept': 'need.Im nineteen years'}, {'sentence': 'I was a loner who read wrote and gamed in my spare time rather than hang out with friends.', 'kp': ['loner who read', 'read wrote', 'wrote and gamed', 'spare time', 'friends'], 'stance': 'CON', 'aspect': 'loner who read', 'topic': None, 'concept': 'loner who read'}, {'sentence': 'People instantly expected me to talk with a lisp walk wi

In [ ]:
# for i, j in zip(retrieved_ranked, sample):
#     # Add counter to the dictionary (implicitly, i)
#     i["counter"] = j["counter"]
#     fout.write(json.dumps(i))
#     fout.write("\n")

In [ ]:
# Working Loop
# for i in mined_counters_:
#     test = {
#         "test": [j for j in i["counter"]]
#     }